In [1]:
import random

In [2]:
print('I want ?X'.split())

['I', 'want', '?X']


In [3]:
def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha()for s in pat[1:])

In [4]:
def pat_match(pattern, saying):
    if is_variable(pattern[0]): return True
    else:
        if pattern[0] != saying[0]: return False
        else:
            return pat_match(pattern[1:], saying[1:])

In [5]:
pat_match('I want ?X'.split(), "I want holiday".split())

True

In [6]:
def pat_match(pattern, saying):
    if is_variable(pattern[0]):
        return pattern[0], saying[0]
    else:
        if pattern[0] != saying[0]: return False
        return pat_match(pattern[1:], saying[1:])

In [7]:
pattern = 'I want ?X'.split()
saying = 'I want holiday'.split()

In [8]:
pat_match(pattern, saying)

('?X', 'holiday')

In [9]:
pat_match("?X equals ?X".split(), "2+2 equals 2+2".split())

('?X', '2+2')

In [10]:
def pat_match(pattern, saying):
    if not pattern or not saying: return []
    
    if is_variable(pattern[0]):
        return [(pattern[0], saying[0])] + pat_match(pattern[1:], saying[1:])
    else:
        if pattern[0] != saying[0]: return []
        else:
            return pat_match(pattern[1:], saying[1:])

In [11]:
pat_match("?X greater than ?Y".split(), "3 greater than 2".split())

[('?X', '3'), ('?Y', '2')]

In [12]:
def pat_to_dict(patterns):
    return {k:v for k, v in patterns}

In [13]:
def subsitite(rule, parsed_rules):   #rule是句子，# parsed_rules是字典
    if not rule: return []
    
    return[parsed_rules.get(rule[0], rule[0])] + subsitite(rule[1:], parsed_rules)

In [14]:
# a = pat_to_dict(pat_match("?X greater than ?Y".split(), "3 greater than 2".split()))

In [15]:
# print(a.get("what if you mean if you got a ?X".split()[8]))

In [16]:
got_patterns = pat_match("I want ?X".split(), "I want iPhone".split())

In [17]:
got_patterns

[('?X', 'iPhone')]

In [18]:
subsitite("what if you mean if you got a ?X".split(), pat_to_dict(got_patterns))

['what', 'if', 'you', 'mean', 'if', 'you', 'got', 'a', 'iPhone']

In [19]:
john_pat = pat_match('?P needs ?X'.split(), "John needs resting".split())

In [20]:
' '.join(subsitite("What if you mean if you got a ?X".split(), pat_to_dict(got_patterns)))

'What if you mean if you got a iPhone'

In [21]:
' '.join(subsitite("Why does ?P needs ?X".split(), pat_to_dict(john_pat)))

'Why does John needs resting'

In [22]:
defined_patterns = {
    "I need ?X": ["Image you will get ?X soon", "why do you need ?X ?"],
    "My ?X told me something": ["Talk about more about your ?X", "How do you think about your ?X ?"]
}

In [23]:
for i in defined_patterns:
    print(pat_match(i.split(), "I need iPhone".split()) != [])

True
False


In [24]:
def get_response(saying, rules = defined_patterns):
    for i in list(rules.keys()):
        if pat_match(i.split(), saying.split()) != []:
            match_key = i
            match = pat_match(i.split(), saying.split())
            break
    response = ' '.join(subsitite(random.choice(rules[match_key]).split(), pat_to_dict(match)))
    return response

In [25]:
get_response('I need iPhone')

'Image you will get iPhone soon'

In [26]:
get_response('My mother told me something')

'How do you think about your mother ?'

## **Segment Match**
我们上边的这种形式，能够进行一些初级的对话了，但是我们的模式逐字逐句匹配的， "I need iPhone" 和 "I need ?X" 可以匹配，但是"I need an iPhone" 和 "I need ?X" 就不匹配了，那怎么办？

为了解决这个问题，我们可以新建一个变量类型 "?*X", 这种类型多了一个星号(*),表示匹配多个

首先，和前文类似，我们需要定义一个判断是不是匹配多个的variable

In [27]:
def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])

In [28]:
is_pattern_segment('?*P')

True

In [29]:
from collections import defaultdict

In [30]:
fail = [True, None]

def pat_match_with_seg(pattern, saying):
    if not pattern or not saying: return []
    
    pat = pattern[0]
    
    if is_variable(pat):
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):
        match, index = segment_match(pattern, saying)
        return [match] + pat_match_with_seg(pattern[1:], saying[index:])
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail

In [31]:
def segment_match(pattern, saying):
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')

    if not rest: return (seg_pat, saying), len(saying)    
    
    for i, token in enumerate(saying):
        if rest[0] == token and is_match(rest[1:], saying[(i + 1):]):
            return (seg_pat, saying[:i]), i
    
    return (seg_pat, saying), len(saying)

def is_match(rest, saying):
    if not rest and not saying:
        return True
    if not all(a.isalpha() for a in rest[0]):
        return True
    if rest[0] != saying[0]:
        return False
    return is_match(rest[1:], saying[1:])

In [32]:
segment_match('?*P is very good'.split(), "My dog and my cat is very good".split())

(('?P', ['My', 'dog', 'and', 'my', 'cat']), 5)

In [33]:
pat_match_with_seg('?*P is very good and ?*X'.split(), "My dog is very good and my cat is very cute".split())

[('?P', ['My', 'dog']), ('?X', ['my', 'cat', 'is', 'very', 'cute'])]

In [34]:
response_pair = {
    'I need ?X': [
        "Why do you need ?X"
    ],
    "I don't like my ?X":["What bad thing did ?X do for you?"]
    
}

In [77]:
pat_match_with_seg('I need ?*X'.split(), "I need an iPhone".split())

[('?X', ['an', 'iPhone'])]

In [36]:
subsitite("Why do you neeed ?X".split(), pat_to_dict(pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())))

['Why', 'do', 'you', 'neeed', ['an', 'iPhone']]

In [37]:
def pat_to_dict(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in patterns}

In [38]:
subsitite("Why do you neeed ?X".split(), pat_to_dict(pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())))

['Why', 'do', 'you', 'neeed', 'an iPhone']

In [39]:
("?*X hello ?*Y", "Hi, how do you do")

('?*X hello ?*Y', 'Hi, how do you do')

In [40]:
subsitite("Hi, how do you do".split(), pat_to_dict(pat_match_with_seg('?*X hello ?Y'.split(), "I am mike, hello".split())))

['Hi,', 'how', 'do', 'you', 'do']

In [41]:
!pip install jieba

# 问题1

In [90]:
rules = {
    "?*X hello ?*Y": ["Hi, how do you do?"],
    "I was ?*X": ["Were you really ?X ?", "I already knew you were ?X ."]
}

In [199]:
def get_response(saying, rules = rules):
    for i in rules:
        match = pat_match_with_seg(i.split(), saying.split())
        if match:
            # 当？*X在位于模式匹配的开头时的情况，这时要判断是否有需要替换的部分
            if not(is_pattern_segment(i.split()[0]) == True and len(match[0][1]) == len(saying.split())):
                    # 随机从模式匹配库中返回一个回答
                    response = ' '.join(subsitite(random.choice(rules[i]).split(), pat_to_dict(match)))
                    return response
                    break

In [200]:
get_response("I was 18 years old" ,rules)

'I already knew you were 18 years old .'

In [201]:
get_response("I am mike, hello", rules)

'Hi, how do you do?'

# 问题2和3

In [208]:
import jieba

In [162]:
ch_rules = {
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想要?y吗？'],
    '?*x讨厌?*y': ['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？'],
    '?*x机器人?*y': ['你为什么要提机器人的事情？', '你为什么觉得机器人要解决你的问题？']
}

In [165]:
jieba.lcut('?*x喜欢?*y')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\zzy\AppData\Local\Temp\jieba.cache
Loading model cost 0.557 seconds.
Prefix dict has been built succesfully.


['?', '*', 'x', '喜欢', '?', '*', 'y']

In [166]:
def cut(string):
    tmp_str = ' '.join(jieba.lcut(string))
    tmp_str = tmp_str.replace("? ", "?").replace("* ", "*")
    return tmp_str.split()

In [167]:
cut('?*x喜欢?*y')

['?*x', '喜欢', '?*y']

In [222]:
def chinese_get_response(saying, rules = ch_rules):
    for i in rules:
        match = pat_match_with_seg(cut(i), cut(saying))
        if match:
            if not(is_pattern_segment(cut(i)[0]) == True and len(match[0][1]) == len(cut(saying))):
                response = ' '.join(subsitite(cut(random.choice(rules.get(i))), pat_to_dict(match)))
                return response
                break

In [226]:
chinese_get_response('我喜欢你')

'你 想要 你 吗 ？'

In [227]:
chinese_get_response('我讨厌睡觉')

'睡觉 有 什么 不好 呢 ？'

In [229]:
chinese_get_response('我写的机器人很棒')

'你 为什么 觉得 机器人 要 解决 你 的 问题 ？'

# 问题4

1.程序优点是方法简介明了，简单，可拓展性强；缺点是当该系统没有用户输入的模式时，将无法匹配；
我觉得可以尝试让机器自动生成这些模式，而不是手动输入。
2.数据驱动我自己的理解是根据数据去做决策；此程序中以模式匹配库中的模式以作为数据驱动的方式。
3.据我自己的了解，数据对于AI的发展来说十分重要，一个好的深度/机器学习模型需要大量数据去训练，
即使有再优秀的模型，没有数据也是白搭。